No âmbito da cadeira de BDCC foi-nos fornecido o ficheiro EVENTS.csv.gz que contém registos de vários pacientes coletados durante a sua estadia na unidade de cuidados intensivos.

O objetivo principal deste trabalho consiste na criação de um modelo de machine learning capaz de prever a duração da estadia dado os registos recolhidos durante essa estadia.

O trabalho está divido em duas partes essenciais, a análise exploratória de dados e a criação do modelo de previsão, sendo que o sucesso da primeira parte pode influênciar significativamente o sucesso da segunda.
Devido á alta dimensionalidade do ficheiro em questão, decidi seguir uma das sugestões propostas pela Professora ecarregar o ficheiro para uma tabela BigQuery do Google Cloud Storage, que tem capacidade de suportar ficheiros desta dimensão. 

Desta forma, posso fazer parte do pre-processamento através de queries SQL, extrair essas views e trabalhar com ficheiros de menor tamanho.

Comecei então por carregar os pacotes que irei utilizar na parte da análise exploratória: O pandas e numpy para pré-processamento e o ploty para visualização.

In [ ]:
# Load the required packages
import pandas as pd  
import numpy as np   
import plotly.graph_objs as go
from plotly.offline import iplot

De seguida carreguei a partir do meu Google Drive os ficheiros resultantes de queries feitas através da Web Interface ao ficheiro original:

In [ ]:
EVENTS_PER_ITEMID = pd.read_csv("/content/drive/My Drive/EVENTS_PER_ITEMID.csv",delimiter=",")
N_PATIENTS_PER_N_STAYS = pd.read_csv("/content/drive/My Drive/N_PATIENTS_PER_N_STAYS.csv",delimiter=",")
ICUSTAYS = pd.read_csv("/content/drive/My Drive/ICUSTAYS.csv",delimiter=",")

A tabela "EVENTS_PER_ITEMID" possui informação relativa ao número de medições feitas para cada item presente no ficheiro original

In [ ]:
EVENTS_PER_ITEMID.head()

,ITEMID,COUNT
0,6240,1
1,7041,1
2,1165,1
3,1747,1
4,1408,1


A tabela "N_PATIENTS_PER_N_STAYS" possui a distribuição de pacientes pelo número total de estaidas na unidade de cuidados intensivos

In [ ]:
N_PATIENTS_PER_N_STAYS.head()

,STAYS,N_PATIENTS
0,0,29
1,1,38002
2,2,5624
3,3,1535
4,4,621


Como alternativa a calcular a diferença entre as datas da ultima mediação e primeira medição, de modo a obter a duração da estadia, decidi utilizar a tabela "ICUSTAYS" também disponibilizada pela professora, que já possui uma coluna com essa informação


In [ ]:
ICUSTAYS.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


De modo a reduzir a dimensionalidade do dados para que estes consigam ser trabalhados, decidi utilizar apenas os items com um maior numero de registos, tendo imposto um valor mínimo de 1000000.

In [ ]:
#Remove items that do not meet the 1000000 event count
EVENTS_PER_ITEMID= EVENTS_PER_ITEMID.drop(EVENTS_PER_ITEMID[EVENTS_PER_ITEMID.COUNT < 1000000].index)

#Plot the respective barchart acording to the dataframe
plot_data = [
    go.Bar(
        x=EVENTS_PER_ITEMID['ITEMID'],
        y=EVENTS_PER_ITEMID['COUNT'],
    )
]

plot_layout = go.Layout(title = 'Number of Events per ITEMID',
                   xaxis = {"type": "category"},
                   xaxis_title = "ITEMID",
                   yaxis_title = "Number of Events")

fig = go.Figure(data=plot_data, layout=plot_layout)
iplot(fig)

De seguida, decidi visualizar o a distribuição de pacientes por número de estadias na unidade de cuidados intensivos

In [ ]:
#Plot a barchart repreesenting the distribution of patients by number of ICU Stays
plot_data = [
    go.Bar(
        x=N_PATIENTS_PER_N_STAYS['STAYS'],
        y=N_PATIENTS_PER_N_STAYS['N_PATIENTS'],
    )
]

plot_layout = go.Layout(title = 'Number of Patients per Number of ICU Stays',
                   xaxis = {"type": "category"},
                   xaxis_title = "Number of ICU Stays",
                   yaxis_title = "Number of Patients")

fig = go.Figure(data=plot_data, layout=plot_layout)
iplot(fig)

Como podemos observar no gráfico anterior (apesar de ser dificil devido á escala), existem 29 pacientes cujas medições não têm associado um ICUSTAY_ID, tornado essa informação irrelevante, e como tal, deverá ser removida


De seguida, utilizando a tabela "ICUSTAYS", visualizei a distribuição da nossa variável alvo, a duração da estadia

In [ ]:
import datetime

#Create a new Pandas DF with the only columns I will need in the following steps (ICUSTAY_ID and LOS)
NEW_ICUSTAYS = ICUSTAYS.filter(['ICUSTAY_ID','LOS'], axis=1)
NEW_ICUSTAYS.columns = ['ICUSTAY_ID', 'TIME']

#Group number of ICU stays by LOS
TIME_ICUSTAYS = NEW_ICUSTAYS.round(0)
TIME_ICUSTAYS = TIME_ICUSTAYS.groupby('TIME')['ICUSTAY_ID'].count().reset_index(name="COUNT")

#Plot a barchart that represents the previous dataframe
plot_data = [
    go.Bar(
        x=TIME_ICUSTAYS['TIME'],
        y=TIME_ICUSTAYS['COUNT'],
    )
]

plot_layout = go.Layout(title = 'Distribution of the Length of ICU Stay',
                   xaxis = {"type": "category"},
                   xaxis_title = "Length of ICU Stay (in days)",
                   yaxis_title = "Number of Stays")

fig = go.Figure(data=plot_data, layout=plot_layout)
iplot(fig)

Como se pode ver, existem estadias que não cheagam a durar um dia na unidade. Como tal, de modo a tentar melhorar a fiebilidade das medições recolhidas, decidi remover essa estadias do dataset final a ser fornecido ao modelo. Não acredito que seriam relevantes para o problema em questão

In [ ]:
from google.colab import files

NEW_ICUSTAYS = NEW_ICUSTAYS.drop(NEW_ICUSTAYS[NEW_ICUSTAYS.TIME < 1].index)
#NEW_ICUSTAYS.to_csv('NEW_ICUSTAYS.csv')
#files.download('NEW_ICUSTAYS.csv')

Depois desta análise, através de uma query ao ficheiro original, obtive uma tabela que cumpre as restrições mencionadas anteriomente que possui o valor mínimo, máximo e médio de um determinado item para uma determinada estadia. 

In [ ]:
DATA = pd.read_csv("/content/drive/My Drive/DATA.csv",delimiter=",")
DATA.head()

,ICUSTAY_ID,ITEMID,MIN,MAX,AVG
0,283784,5819,4.0,8.0,6.854545
1,244961,5819,2.0,10.0,6.706999
2,298603,5819,5.0,8.0,6.771277
3,287432,5819,4.0,8.0,5.973214
4,205283,5819,5.0,8.0,7.940789


Com base na tabela anterior, criei uma nova tabela que para cada ICUSTAY_ID possui os dados de todos os items observados nessa estadia

In [ ]:
#Create a new Pandas Dataframe with the column "ICUSTAY_ID" based on all unique ICUSTAY_IDs from the previous df
RESULT = pd.DataFrame({'ICUSTAY_ID': DATA['ICUSTAY_ID'].unique()})

#For every unique item in the previous df, create 3 columns in the new df that hold information about the Min, Max and Avg of each item
#Initialize those column values at 0
for item in DATA['ITEMID'].unique():
  RESULT[str(item) + '_MIN'] = 0.0
  RESULT[str(item) + '_MAX'] = 0.0
  RESULT[str(item) + '_AVG'] = 0.0

#Set ICUSTAY_ID as the dataframe's index
RESULT = RESULT.set_index('ICUSTAY_ID')

#Iterate over the "DATA" dataframe and for each row:
for index, row in DATA.iterrows():
    item = row["ITEMID"]
    string1 = str(int(item)) + '_MIN'
    string2 = str(int(item)) + '_MAX'
    string3 = str(int(item)) + '_AVG'
    #Replace the specific item column with the correct measurement for that column
    RESULT.at[int(row["ICUSTAY_ID"]), string1] = row["MIN"]
    RESULT.at[int(row["ICUSTAY_ID"]), string2] = row["MAX"]
    RESULT.at[int(row["ICUSTAY_ID"]), string3] = row["AVG"]


RESULT.head()

,5819_MIN,5819_MAX,5819_AVG,8554_MIN,8554_MAX,8554_AVG,618_MIN,618_MAX,618_AVG,8549_MIN,8549_MAX,8549_AVG,646_MIN,646_MAX,646_AVG,8553_MIN,8553_MAX,8553_AVG,8368_MIN,8368_MAX,8368_AVG,5817_MIN,5817_MAX,5817_AVG,3603_MIN,3603_MAX,3603_AVG,220050_MIN,220050_MAX,220050_AVG,220045_MIN,220045_MAX,220045_AVG,220277_MIN,220277_MAX,220277_AVG,220210_MIN,220210_MAX,220210_AVG,220181_MIN,...,51_AVG,211_MIN,211_MAX,211_AVG,456_MIN,456_MAX,456_AVG,834_MIN,834_MAX,834_AVG,8551_MIN,8551_MAX,8551_AVG,8532_MIN,8532_MAX,8532_AVG,5820_MIN,5820_MAX,5820_AVG,5815_MIN,5815_MAX,5815_AVG,581_MIN,581_MAX,581_AVG,8441_MIN,8441_MAX,8441_AVG,3609_MIN,3609_MAX,3609_AVG,3450_MIN,3450_MAX,3450_AVG,8518_MIN,8518_MAX,8518_AVG,742_MIN,742_MAX,742_AVG
ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
283784,4.0,8.0,6.854545,100.0,100.0,100.000000,8.0,27.0,19.631579,120.0,120.0,120.000000,97.0,100.0,99.666667,30.0,30.0,30.000000,0.0,0.0,0.000000,100.0,130.0,129.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,56.0,90.0,69.781818,80.000000,110.000,91.693878,0.0,0.0,0.000000,160.0,180.0,179.636364,0.0,0.0,0.0,90.0,92.0,90.690909,50.0,60.0,52.000000,0.000000,0.000000,0.000000,55.0,101.0,68.703704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
244961,2.0,10.0,6.706999,100.0,100.0,100.000000,0.0,34.0,18.970134,110.0,120.0,110.273159,90.0,100.0,97.634843,25.0,35.0,30.207592,20.0,85.0,56.104990,80.0,130.0,104.368030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,149.511435,37.5,140.0,64.003593,45.666698,102.667,79.511124,98.0,98.0,98.000000,180.0,190.0,187.434944,0.0,0.0,0.0,90.0,92.0,91.551069,35.0,60.0,45.249406,76.199997,87.500000,80.722808,29.0,78.0,49.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
298603,5.0,8.0,6.771277,10.0,100.0,91.861702,6.0,27.0,15.133690,130.0,160.0,137.872340,55.0,100.0,96.949438,30.0,30.0,30.000000,37.0,142.0,65.267442,85.0,90.0,85.930851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,112.482558,72.0,159.0,96.349462,45.000000,107.000,74.802485,96.0,98.0,97.333333,160.0,170.0,160.904255,0.0,0.0,0.0,90.0,90.0,90.000000,50.0,60.0,58.138298,98.800003,98.800003,98.800003,32.0,100.0,57.851852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
287432,4.0,8.0,5.973214,100.0,100.0,100.000000,5.0,30.0,13.432203,120.0,120.0,120.000000,86.0,100.0,99.064655,30.0,40.0,33.273543,32.0,96.0,63.986301,90.0,90.0,90.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,119.657534,73.0,136.0,90.963415,49.000000,101.000,74.840237,0.0,0.0,0.000000,160.0,160.0,160.000000,0.0,0.0,0.0,92.0,93.0,92.197309,50.0,60.0,52.008929,0.000000,0.000000,0.000000,0.0,86.0,62.959538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
205283,5.0,8.0,7.940789,100.0,100.0,100.000000,9.0,27.0,19.822368,120.0,120.0,120.000000,68.0,100.0,96.597561,30.0,30.0,30.000000,31.0,64.0,49.013986,90.0,100.0,99.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,130.426573,64.0,88.0,73.276316,59.333302,79.000,70.305541,0.0,0.0,0.000000,150.0,160.0,156.533333,0.0,0.0,0.0,60.0,90.0,89.407895,30.0,60.0,59.407895,62.099998,73.500000,65.989411,27.0,46.0,38.416667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


De seguida fiz um join da tabela resultante com a "ICUSTAYS" na coluna "ICUSTAY_ID" de modo a associar o tempo da estadia com a informação recolhida.
Para finalizar o préprocessamento, decidi tornar este problema num problema de  classificação binária e converti a duração da estaida para 2 categorias: "Short Stay" (<10 dias) e "Long Stay" (>= 10 dias)

Na criação do modelo de previsão, comecei por importar os recursos que iria utilizar do pacote sklearn.

Comecei por transformar as features através do MinMaxScaler, num alcance entre 0 e 1. A Normalização de um conjunto de dados é um requisito comum para estimadores de machine learning e estes podem funcionar mal se este passo não for cumprido.

De seguida decidi ainda fazer um pouco de feature selection para tentar combater o overfitting, utilizando o DecisionTreeClassifier e o SelectFromModel com a mediana como threshold.

Depois destes passos, passo para a criação de modelos de LogisticRegression, KNN, DecisionTree, SVM e NaiveBays e por um modelo de stacking ensemble dos modelos anteriores. Por fim, comparo a sua performance para a tarefa dada.



In [ ]:
import warnings
warnings.filterwarnings("ignore")

#Import machine learning packages/resources
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime


#Merge the "RESULT" dataframe with the "NEW_ICUSTAYS" dataframe to associate a LOS (TIME) column to the "RESULT dataframe"
RESULT = pd.merge(RESULT, NEW_ICUSTAYS, on='ICUSTAY_ID', how='inner')

#Create a column "TIME_C" based on the "TIME" column that converts the numeric value into a categorical one 
RESULT['TIME_C'] = 'Short Stay'
RESULT.loc[(RESULT['TIME'] > 10), 'TIME_C'] = 'Long Stay'

#Separate the target variable and the input variables
x = RESULT.drop(['ICUSTAY_ID','TIME','TIME_C'],axis = 1).values
y = RESULT['TIME_C'].values

#Apply a MinMaxScaler normalization to the input variable set
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)

#Apply Feature selection to the input varibale set using a DecisionTreeClassifier with 'median' as threshold
clf = DecisionTreeClassifier()
trans = SelectFromModel(clf, threshold='median')
x_trans = trans.fit_transform(x, y)

print("We started with {0} features but retained only {1} of them".format(x.shape[1] - 1, x_trans.shape[1]))

#Split the dataset into train and test data
x_train, x_validation, y_train, y_validation = train_test_split(x_trans, y, test_size=0.20, random_state=1)


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
 
# get a stacking ensemble of models
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('lr', LogisticRegression()))
	level0.append(('knn', KNeighborsClassifier()))
	level0.append(('cart', DecisionTreeClassifier()))
	level0.append(('svm', SVC()))
	level0.append(('bayes', GaussianNB()))
	# define meta learner model
	level1 = LogisticRegression()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
	return model
 
# get a list of models to evaluate
def get_models():
	models = dict()
	models['lr'] = LogisticRegression()
	models['knn'] = KNeighborsClassifier()
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	models['stacking'] = get_stacking()
	return models
 
#evaluate a given model using cross-validation
def evaluate_model(model,x,y):
	#Use 5-fold cross validation with 3 repeats
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#Use 'accuracy' as the scoring metric and n_jobs as -1 to allow the use of all of the machine's cores and maximage their usage
	scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores
 
# get the models to evaluate
models = get_models()

# evaluate the models and store results
for name, model in models.items():
	scores = evaluate_model(model,x_train,y_train)
	print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

We started with 104 features but retained only 53 of them
>lr 0.906 (0.002)
>knn 0.907 (0.002)
>cart 0.887 (0.003)
>svm 0.914 (0.002)
>bayes 0.542 (0.104)
>stacking 0.916 (0.002)


Podemos ver que o modelo de ensamble teve uma performance superior a todos os outros, o que é de esperar, com perto de 92% de accuracy. 

Vamos agora então utiliza-lo para fazer predictions para o validation set.

In [ ]:
# Make predictions with the ensemble model on validation dataset
model = get_stacking()
model.fit(x_train, y_train)
predictions = model.predict(x_validation)

Podemos agora avaliar as predictions comparando-as com o resultado esperado no validation set e de seguida calcular a accuracy, a confusion matrix e produzir um classification report.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Evaluate predictions
print(accuracy_score(y_validation, predictions))
print(confusion_matrix(y_validation, predictions))
print(classification_report(y_validation, predictions))

0.9197283112071627
[[ 764  586]
 [ 194 8173]]
              precision    recall  f1-score   support

   Long Stay       0.80      0.57      0.66      1350
  Short Stay       0.93      0.98      0.95      8367

    accuracy                           0.92      9717
   macro avg       0.87      0.77      0.81      9717
weighted avg       0.91      0.92      0.91      9717

